In [ ]:
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests as request
import pandas as pd
import pymongo

In [ ]:
#Use beautifulsoup to parse the site 
url = 'https://mars.nasa.gov/news/'
response = request.get(url)
soup = bs(response.text,'html.parser')

In [ ]:
# print(soup.prettify())

In [ ]:
#Gather most recent title and article description
results = soup.find(class_ = "slide")
description = soup.find(class_ = "rollover_description_inner")
title = soup.find(class_ = "content_title")

news_title = title.text.replace('\n','').replace('\r','')
news_p = description.text.replace('\n','').replace('\r','')

In [ ]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_response = request.get(jpl_url)
jpl_soup = bs(jpl_response.text,'html.parser')

In [ ]:
# print(jpl_soup.prettify())

In [ ]:
#Gather image urls from the jpl site
jpl = jpl_soup.find_all(class_ = 'img')
url_list = []
for j in jpl:
    pl = j.find('img')['src']
    img_urls = jpl_url + pl
    url_list.append(img_urls)
featured_image_url = url_list[0]

In [ ]:
#Visit the mars twitter page and gather the latest weather content
mars_tweet_url = 'https://twitter.com/marswxreport?lang=en'
mars_response = request.get(mars_tweet_url)
tweet_soup = bs(mars_response.text,'html.parser')
# print(tweet_soup.prettify())
tweets = tweet_soup.find_all(class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
wx = []
for t in tweets:
    if 'daylight' in t.text:
        wx.append(t.text)
mars_weather = wx[0]

In [ ]:
#Visit mars space facts page and gather table data
mars_facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(mars_facts_url)

df = tables[0]
cols = list(df[0])
data = list(df[1])

mars_dict = {'description':cols,'value':data}
mars_df = pd.DataFrame(mars_dict)
mars_df.set_index('description',inplace=True)
mars_table = mars_df.to_html()
#table stored as an html string with \n removed
mars_string = mars_table.replace('\n','')

In [ ]:
# Mars Hemispheres
hemisphere_image_urls = [
    {"title": "Cerberus Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url":"https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg"},
    {"title": "Valles Marineris Hempisphere", "img_url":"https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg"}
    ]

In [ ]:
# Insert mars hemisphere information in to a Mongo DB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_db
collections = db.items
for x in hemisphere_image_urls:
    collections.insert_one(x)

In [ ]:
# Query the mars_db
mars_query = db.items.find()
for m in mars_query:
    print(m)

In [ ]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)
browser = init_browser()

In [ ]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_url = "https://astrogeology.usgs.gov"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')
url_list = []
title_list =[]
jpg_url_list =[]
hemisphere_image_urls = []
results = soup.find('div', class_="collapsible results")
links = results.find_all('a', class_="item product-item")
titles= results.find_all('h3')
for link in links:
    url_list.append(base_url+link["href"])

for title in titles:
    title_list.append(title.text)

for item in url_list:
    browser.visit(item)
    time.sleep(1)

    html = browser.html
    soup = BeautifulSoup(html, 'lxml')

    downloads = soup.find('div', class_="downloads")
    jpg = downloads.find_all('a')
    jpg_url = jpg[0]["href"]
    jpg_url_list.append(jpg_url)

for i in range(0,len(title_list)):
    hemisphere_image_urls.append({"title": title_list[i], "img_url": jpg_url_list[i]})

In [ ]:
hemisphere_image_urls